In [147]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import numpy as np

In [71]:
#Drop any songs that have NaN as a genre
genres = pd.read_csv('fma_metadata/tracks.csv', low_memory=False).dropna(subset=['track.7'])
genres = genres.rename(index=str, columns={'Unnamed: 0': 'id', 'track.7': 'genre'})

In [72]:
#Reduce size of df to just id and genre
genres = genres[['id', 'genre']]

In [75]:
#Get 25,000 of the songs that don't have NaN as a genre
genres = genres[1:].sample(25000)

In [88]:
genres = genres.set_index('id')
genres.head()

,genre
id,
93953,Rock
20086,Old-Time / Historic
108626,Experimental
32776,Electronic
82250,Experimental


In [84]:
features = pd.read_csv('fma_metadata/features.csv', low_memory=False)
features = features.rename(index=str, columns={'feature': 'id'})

In [91]:
#filter out the same track IDs
features = features.loc[features.id.isin(genres.index.values)].set_index('id')

In [92]:
features.head()

,chroma_cens,chroma_cens.1,chroma_cens.2,chroma_cens.3,chroma_cens.4,chroma_cens.5,chroma_cens.6,chroma_cens.7,chroma_cens.8,chroma_cens.9,...,tonnetz.39,tonnetz.40,tonnetz.41,zcr,zcr.1,zcr.2,zcr.3,zcr.4,zcr.5,zcr.6
id,,,,,,,,,,,,,,,,,,,,,
3,1.8889633417e+00,7.6053929329e-01,3.4529656172e-01,2.2952005863e+00,1.6540306807e+00,6.7592434585e-02,1.3668476343e+00,1.0540937185e+00,1.0810308903e-01,6.1918509007e-01,...,6.3831120729e-02,1.4211839065e-02,1.7740072682e-02,2.8246941566e+00,4.6630859375e-01,8.4578499198e-02,6.3964843750e-02,0.0000000000e+00,1.7167237997e+00,6.9330163300e-02
5,5.2756297588e-01,-7.7654317021e-02,-2.7961030602e-01,6.8588310480e-01,1.9375696182e+00,8.8083887100e-01,-9.2319184542e-01,-9.2723226547e-01,6.6661673784e-01,1.0385463238e+00,...,4.0730185807e-02,1.2690781616e-02,1.4759079553e-02,6.8084154129e+00,3.7500000000e-01,5.3114086390e-02,4.1503906250e-02,0.0000000000e+00,2.1933031082e+00,4.4860601425e-02
134,9.1844475269e-01,6.7414724827e-01,5.7781827450e-01,1.2811170816e+00,9.3374562263e-01,7.8176945448e-02,1.1992042065e+00,-1.7522314191e-01,9.2548191547e-01,1.4385091066e+00,...,5.8766160160e-02,1.6322381794e-02,1.5819497406e-02,4.7310867310e+00,4.1943359375e-01,6.4369551837e-02,5.0781250000e-02,0.0000000000e+00,1.8061059713e+00,5.4622855037e-02
135,9.7840219736e-01,6.2357187271e-01,1.1292605400e+00,-1.0439702272e+00,-1.1272884607e+00,-3.2641757280e-02,-8.7843364477e-01,-9.1623413563e-01,-7.4325716496e-01,6.0923475027e-01,...,1.0427882522e-01,1.6731236130e-02,2.0464096218e-02,-3.8451351225e-02,2.3486328125e-01,5.0837226212e-02,5.0781250000e-02,3.4179687500e-03,2.9787299037e-01,2.4899475276e-02
136,9.1500061750e-01,-6.4347624779e-01,-4.6050721407e-01,-5.3070127964e-01,-3.6446049809e-01,-2.2685964406e-01,-6.0376849025e-02,-2.6767292619e-01,-6.3476271927e-02,2.4851760268e-01,...,7.6808303595e-02,1.7914786935e-02,1.6705930233e-02,5.5877000093e-01,1.4746093750e-01,3.6686290056e-02,3.4179687500e-02,3.4179687500e-03,8.0502045155e-01,1.6904523596e-02


In [93]:
features.reindex(genres.index.values)

,chroma_cens,chroma_cens.1,chroma_cens.2,chroma_cens.3,chroma_cens.4,chroma_cens.5,chroma_cens.6,chroma_cens.7,chroma_cens.8,chroma_cens.9,...,tonnetz.39,tonnetz.40,tonnetz.41,zcr,zcr.1,zcr.2,zcr.3,zcr.4,zcr.5,zcr.6
id,,,,,,,,,,,,,,,,,,,,,
93953,-1.5874287486e-01,-5.9277850389e-01,-3.9923959970e-01,-2.5748467445e-01,-2.2578692436e-01,-2.6395587251e-02,2.4995682240e+00,1.8951414824e+00,4.1370522976e-01,-3.3995303512e-01,...,5.8169201016e-02,1.9423143938e-02,1.7380004749e-02,1.9453805542e+02,7.7929687500e-01,3.7423402071e-02,2.9785156250e-02,3.4179687500e-03,9.8379526138e+00,3.3667922020e-02
20086,-3.7079554796e-01,-1.7796246707e-01,-5.0857496262e-01,-3.5529714823e-01,-2.8024035692e-01,-5.2816200256e-01,-5.8888262510e-01,3.8071337342e-01,1.0215874016e-01,2.4042625725e-01,...,1.2132487446e-01,2.4534692988e-02,2.3457542062e-02,1.0027783966e+02,3.9257812500e-01,5.3145077080e-02,5.2246093750e-02,0.0000000000e+00,5.9697732925e+00,1.7180748284e-02
108626,-5.7287716866e-01,-1.0494841337e+00,-1.0454161167e+00,-1.2939493656e+00,-8.8696432114e-01,9.9912440777e-01,2.9571924210e+00,4.1035461426e+00,4.1013078690e+00,7.1374140680e-02,...,1.0699443519e-01,1.8340403214e-02,2.6030596346e-02,3.7762420177e+00,4.1357421875e-01,4.5061796904e-02,3.3691406250e-02,0.0000000000e+00,1.6431454420e+00,3.8413606584e-02
32776,-1.0992000103e+00,-1.3128474951e+00,-9.4788420200e-01,3.7686908245e-01,2.1139997244e-01,-3.9794772863e-01,-4.6096768975e-01,-3.8589403033e-01,3.6123901606e-01,-2.1094664931e-01,...,7.2626195848e-02,1.5987791121e-02,1.7749791965e-02,6.4292490482e-01,5.7714843750e-01,1.7851457000e-01,1.7919921875e-01,0.0000000000e+00,2.6403039694e-01,8.0036029220e-02
82250,1.3813786507e+00,1.3853936195e+00,1.9287285805e+00,1.6893213987e+00,2.0497715473e+00,1.5189225674e+00,2.5142071247e+00,2.0595555305e+00,2.1993477345e+00,1.3730596304e+00,...,7.2355799377e-02,1.6297915950e-02,1.8117737025e-02,-5.3211545944e-01,7.0751953125e-01,2.1427534521e-01,1.9873046875e-01,0.0000000000e+00,4.7845137119e-01,1.3050070405e-01
130639,2.6179051399e-01,-7.5792305171e-02,-7.0668959618e-01,1.3321990967e+00,-8.9946824312e-01,-1.0267541409e+00,-9.5349943638e-01,9.0396785736e-01,1.8090956211e+00,-3.3172917366e-01,...,9.1820351779e-02,1.6073934734e-02,2.4608863518e-02,6.6841335297e+00,4.0332031250e-01,5.3696155548e-02,2.8808593750e-02,3.9062500000e-03,2.2833302021e+00,5.5931042880e-02
57983,-5.8587020636e-01,2.4682320654e-01,-4.6796211600e-01,-5.6815713644e-01,9.3867599964e-01,-4.1668754816e-01,5.9253686666e-01,-7.7173632383e-01,-8.3341342211e-01,-8.5945940018e-01,...,7.1509018540e-02,1.9288280979e-02,2.3950980976e-02,4.6157867432e+01,5.4345703125e-01,4.0402315557e-02,3.0761718750e-02,0.0000000000e+00,5.2917671204e+00,3.9432965219e-02
3911,-7.8936213255e-01,-1.2030998468e+00,-1.2867045403e+00,-6.6311138868e-01,-7.3343783617e-01,-7.8307104111e-01,-4.6657964587e-01,-7.9180926085e-01,-9.8169612885e-01,-4.7835335135e-01,...,1.4213909209e-01,2.6115613058e-02,2.6961628348e-02,1.5862684631e+02,2.9052734375e-01,1.9128674641e-02,1.7578125000e-02,4.8828125000e-04,8.1485996246e+00,1.1354391463e-02
17492,-4.4171807170e-01,-8.2176303864e-01,-8.5352426767e-01,-2.9371833801e-01,-1.0421884060e+00,-4.6237495542e-01,-6.6695809364e-01,9.0923406184e-02,-3.2917076349e-01,-1.2510716915e-01,...,7.8870743513e-02,1.5943966806e-02,1.5549796633e-02,7.9809899330e+00,4.1894531250e-01,6.3676856458e-02,5.4199218750e-02,0.0000000000e+00,2.0853135586e+00,4.1035722941e-02


In [106]:
features = features.merge(genres, on='id')

## First trying without any feature reduction

In [148]:
training_set = features[:19922]
validation_set = features[19922:22427]
test_set = features[22427:]
train_X = training_set.drop('genre', axis=1)
train_Y = training_set['genre']
vali_X = scaler.fit_transform(validation_set.drop('genre', axis=1))
vali_Y = validation_set['genre'].values
test_X = scaler.fit_transform(test_set.drop('genre', axis=1))
test_Y = test_set['genre'].values
scaler = StandardScaler()
train_X = scaler.fit_transform(train_X)
logreg = LogisticRegression(C=0.09, multi_class='ovr')
logreg.fit(train_X, train_Y)
print('Training Accuracy: ' + str(logreg.score(train_X, train_Y)))
print('Validation Accuracy: ' + str(logreg.score(vali_X, vali_Y)))
print('Test Accuracy: ' + str(logreg.score(test_X, test_Y)))

/Users/Jesse/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/data.py:617: DataConversionWarning: Data with input dtype object were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/Users/Jesse/anaconda3/lib/python3.6/site-packages/sklearn/base.py:462: DataConversionWarning: Data with input dtype object were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
/Users/Jesse/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/data.py:617: DataConversionWarning: Data with input dtype object were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/Users/Jesse/anaconda3/lib/python3.6/site-packages/sklearn/base.py:462: DataConversionWarning: Data with input dtype object were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
/Users/Jesse/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/data.py:617: DataConversionWarning: Data with i

Training Accuracy: 0.6884348960947696
Validation Accuracy: 0.5604790419161677
Test Accuracy: 0.6404974737660318


## Now with PCA

In [160]:
train_X = training_set.drop('genre', axis=1)
train_Y = training_set['genre']
vali_X = scaler.fit_transform(validation_set.drop('genre', axis=1))
vali_Y = validation_set['genre'].values
test_X = scaler.fit_transform(test_set.drop('genre', axis=1))
test_Y = test_set['genre'].values
pca = PCA(n_components=500)
principal = pca.fit(train_X)

/Users/Jesse/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/data.py:617: DataConversionWarning: Data with input dtype object were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/Users/Jesse/anaconda3/lib/python3.6/site-packages/sklearn/base.py:462: DataConversionWarning: Data with input dtype object were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
/Users/Jesse/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/data.py:617: DataConversionWarning: Data with input dtype object were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/Users/Jesse/anaconda3/lib/python3.6/site-packages/sklearn/base.py:462: DataConversionWarning: Data with input dtype object were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


In [161]:
scaler = StandardScaler()
train_X = pca.transform(scaler.fit_transform(train_X))
vali_X = pca.transform(vali_X)
test_X = pca.transform(test_X)
logreg = LogisticRegression(C=0.09, multi_class='ovr')
logreg.fit(train_X, train_Y)
print('Training Accuracy: ' + str(logreg.score(train_X, train_Y)))
print('Validation Accuracy: ' + str(logreg.score(vali_X, vali_Y)))
print('Test Accuracy: ' + str(logreg.score(test_X, test_Y)))

/Users/Jesse/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/data.py:617: DataConversionWarning: Data with input dtype object were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/Users/Jesse/anaconda3/lib/python3.6/site-packages/sklearn/base.py:462: DataConversionWarning: Data with input dtype object were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
/Users/Jesse/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Training Accuracy: 0.6612287922899307
Validation Accuracy: 0.5608782435129741
Test Accuracy: 0.6335017489312087
